In [1]:
##### ##### ##### ##### ##### ##### #####
# Note that this is the more resource-heavy version due to spectrogram rendering.
# If you're looking for the "lite" version without the spectrogram, see shake_udp_lite.ipynb
##### ##### ##### ##### ##### ##### #####

### Created by Ian Nesbitt, University of Maine, using RaspberryShake UDP documentation:
### https://manual.raspberryshake.org/udp.html

### Live updating plot created using the following as a template:
### https://github.com/anujgupta82/Musings/blob/master/Dynamic%20or%20Live%20update%20of%20a%20Plot.ipynb

In [2]:
%matplotlib widget

In [ ]:
import socket as s
import math as M
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# from https://docs.obspy.org/_modules/obspy/imaging/spectrogram.html#_nearest_pow_2:
def _nearest_pow_2(x):
    """
    Find power of two nearest to x

    >>> _nearest_pow_2(3)
    2.0
    >>> _nearest_pow_2(15)
    16.0

    :type x: float
    :param x: Number
    :rtype: Int
    :return: Nearest power of 2 to x
    """
    a = M.pow(2, M.ceil(np.log2(x)))
    b = M.pow(2, M.floor(np.log2(x)))
    if abs(a - x) < abs(b - x):
        return a
    else:
        return b

ch = 'ENZ'                               # pick your favorite channel
freq = 100                               # sampling frequency in Hz of rshake (1D = 50; 3D & 4D = 100)
seconds = 60                             # number of seconds of data to display
gain = 10                                # higher gain = more spectrogram contrast (value must be positive float)
host = ''                                # blank for localhost (data is being sent here by remote Shake!)
port = 10001                             # Port to bind to

save = False
displ_samps = seconds * freq
wlen = freq / 100
per_lap = 0.9
mult = 8.0
sock = s.socket(s.AF_INET, s.SOCK_DGRAM)
sock.setsockopt(s.SOL_SOCKET, s.SO_REUSEADDR, 1)
sock.setblocking(0)
sock.bind((host, port))

nfft1 = int(_nearest_pow_2(wlen * freq))
nlap1 = int(nfft1 * per_lap)
if mult is not None:
    mult = int(_nearest_pow_2(mult))
    mult = mult * nfft1

#initialize the graph and settings
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(13,10))
#plt.ion()

fig.canvas.draw()

n = 0
stream = []                             # our stream (simple list object)
data = []
curr = datetime.now()
stop = False

while not stop:                                # loop until KeyboardInterrupt
    if stream == []:
        print('Waiting for data on Port:', port)
    try:
        data, addr = sock.recvfrom(1024)    # wait to receive data
    except KeyboardInterrupt:
        stop = True
        pass
    except BlockingIOError:
        pass
    if data != []:
        s = data.decode('UTF-8').strip("'{}").split(', ') # clean and listify the data
        if ch in s[0]:                      # only listen to the specific channel
            if stream == []:
                print(f"Start receiving data from channel {ch}")
            prev = curr                     # timing
            curr = datetime.now()           # timing
            fps = 1/(curr-prev).total_seconds() # timing
            for smp in s[2:]:               # convert strings to ints
                stream.append(int(smp))
            npts = len(stream)
            if npts > displ_samps:          # if the number of samples exceeds the display value, slice the array
                stream = stream[npts - displ_samps:] # move the array to the right, get rid of old samples
                npts = len(stream)
            ax1.clear()                     # ready the plot axis for a new draw
            ax1.set_xlim(0, npts)           # being explicit here helps speed things up slightly
            ax1.set_ylim(min(stream)-25, max(stream)+25)

            nfft = nfft1
            nlap = nlap1
            if nfft > npts:
                nfft = int(_nearest_pow_2(npts / 8.0))
                nlap = int(nfft * per_lap)
            specgram = ax2.specgram(stream, NFFT=nfft, pad_to=mult, Fs=freq, noverlap=nlap)[0] # take only the spectrogram
            ax2.clear()                     # ditch default spectrogram display, ready axis for new image
            
            # here we flip the image upright and apply gain.
            # doing it in 1 step is actually a couple microseconds faster.
            # by default the gain is 10 (=10th root)
            specgram = np.flipud(specgram**(1/gain))

            ax1.set(title='RaspberryShake live UDP output - fps: %.2f' % (fps), ylabel='Counts')
            ax2.set(xlabel='Samples', ylabel='Frequency (Hz)')
            ax2.imshow(specgram, extent=(0,npts,0,freq/2), aspect='auto')
            ax1.plot(stream, linewidth=0.5)
            if save:
                plt.savefig('figs/%05d.png' % n)
                if n == 239:
                    print('6k samples reached; frame full')
                n += 1
            else:
                fig.canvas.draw()
                fig.show()
sock.close()